<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/parsing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Log-based Anomaly Detection with Deep Learning: How Far Are We?](https://colab.research.google.com/drive/13og-BnOONCyEJ2dup6fDstkYY-y23NmC#scrollTo=FlbDlUAoxCWX)

In [4]:
import sys
sys.path.append('/content/')

In [5]:
import Drain

In [19]:
input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'HDFS_2k.log'  # The input log file name
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)
parser.parse(log_file)


Parsing file: /content/HDFS_2k.log
Total size after encoding is 2000 2000
Parsing done. [Time taken: 0:00:00.315274]


In [23]:
input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'BGL_2k.log'  # The input log file name
log_format = '<Label> <Timestamp> <Date> <Node> <Time> <NodeRepeat> <Type> <Component> <Level> <Content>'  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)
parser.parse(log_file)

Parsing file: /content/BGL_2k.log
Total size after encoding is 2000 2000
Parsing done. [Time taken: 0:00:00.543120]


In [28]:
import Spell

input_dir  = '/content/'  # The input directory of log file
output_dir = 'Spell_result/'  # The output directory of parsing results
log_file   = 'HDFS_2k.log'  # The input log file name
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
tau        = 0.5  # Message type threshold (default: 0.5)
regex      = []  # Regular expression list for optional preprocessing (default: [])

parser = Spell.LogParser(indir=input_dir, outdir=output_dir, log_format=log_format, tau=tau, rex=regex)
parser.parse(log_file)

Parsing file: /content/HDFS_2k.log
extracted 1000 log lines from /content/HDFS_2k.log
extracted 2000 log lines from /content/HDFS_2k.log
output result Spell_result/


100%|██████████| 247/247 [00:00<00:00, 49138.79it/s]


Parsing done. [Time taken: 0:00:01.186307]


In [5]:
import sys
sys.path.append('/content/')
import Drain, evaluator2
import os
import pandas as pd


input_dir = '/content/' # The input directory of log file
output_dir = 'Drain_result/' # The output directory of parsing results

benchmark_settings = {
    'HDFS': {
        'log_file': 'HDFS_2k.log',
        'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>',
        'regex': [r'blk_-?\d+', r'(\d+\.){3}\d+(:\d+)?'],
        'st': 0.5,
        'depth': 4
        },

    'BGL': {
        'log_file': 'BGL_2k.log',
        'log_format': '<Label> <Timestamp> <Date> <Node> <Time> <NodeRepeat> <Type> <Component> <Level> <Content>',
        'regex': [r'core\.\d+'],
        'st': 0.5,
        'depth': 4
        },

}

bechmark_result = []
for dataset, setting in benchmark_settings.items():
    print('\n=== Evaluation on %s ==='%dataset)
    indir = os.path.join(input_dir, os.path.dirname(setting['log_file']))
    log_file = os.path.basename(setting['log_file'])

    parser = Drain.LogParser(log_format=setting['log_format'], indir=indir, outdir=output_dir, rex=setting['regex'], depth=setting['depth'], st=setting['st'])
    parser.parse(log_file)

    F1_measure, accuracy = evaluator2.evaluate(
                           groundtruth=os.path.join(indir, log_file + '_structured.csv'),
                           parsedresult=os.path.join(output_dir, log_file + '_structured.csv')
                           )
    bechmark_result.append([dataset, F1_measure, accuracy])


print('\n=== Overall evaluation results ===')
df_result = pd.DataFrame(bechmark_result, columns=['Dataset', 'F1_measure', 'Accuracy'])
df_result.set_index('Dataset', inplace=True)
print(df_result)
df_result.T.to_csv('Drain_bechmark_result.csv')


=== Evaluation on HDFS ===
Parsing file: /content/HDFS_2k.log
Total size after encoding is 2000 2000
Parsing done. [Time taken: 0:00:00.278615]
Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Parsing_Accuracy: 0.9975

=== Evaluation on BGL ===
Parsing file: /content/BGL_2k.log
Total size after encoding is 2000 2000
Parsing done. [Time taken: 0:00:00.254135]
Precision: 0.9992, Recall: 1.0000, F1_measure: 0.9996, Parsing_Accuracy: 0.9625

=== Overall evaluation results ===
         F1_measure  Accuracy
Dataset                      
HDFS       0.999984    0.9975
BGL        0.999599    0.9625


In [35]:
import sys
import pandas as pd
from collections import defaultdict
from scipy.special import comb
df_groundtruth = pd.read_csv('/content/HDFS_2k.log_structured.csv')
df_parsedlog = pd.read_csv('/content/Drain_result/HDFS_2k.log_structured.csv')
# Remove invalid groundtruth event Ids
non_empty_log_ids = df_groundtruth[~df_groundtruth['EventId'].isnull()].index
df_groundtruth = df_groundtruth.loc[non_empty_log_ids]
df_parsedlog = df_parsedlog.loc[non_empty_log_ids]



In [9]:
df_parsedlog['EventId']

0       dc2c74b7
1       dc2c74b7
2       5d5de21c
3       dc2c74b7
4       dc2c74b7
          ...   
1995    bbb51b95
1996    46003790
1997    bbb51b95
1998    dc2c74b7
1999    bbb51b95
Name: EventId, Length: 2000, dtype: object

In [25]:
df_groundtruth.head(5)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203615,148,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_38865049064139...,E10,PacketResponder <*> for block blk_<*> terminating
1,2,81109,203807,222,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-6952295868487...,E10,PacketResponder <*> for block blk_<*> terminating
2,3,81109,204005,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,E6,BLOCK* NameSystem.addStoredBlock: blockMap upd...


In [26]:
df_parsedlog.head(3)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203615,148,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_38865049064139...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]
1,2,81109,203807,222,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-6952295868487...,dc2c74b7,PacketResponder <*> for block <*> terminating,[]
2,3,81109,204005,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap upd...,[]


In [42]:
event = []
for i in df_parsedlog['EventId']:
  if i not in event:
    event.append(i)

In [44]:
event2 = []
for i in df_groundtruth['EventId']:
  if i not in event2:
    event2.append(i)

In [47]:
len(event2)

14

In [61]:
for i in range(len(event)):
  a = df_parsedlog['EventTemplate'][df_parsedlog['EventId']==event[i]]
  for j in a:
    print(j)
    break


PacketResponder <*> for block <*> terminating
BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>
Received block <*> of size <*> from /<*>
Receiving block <*> src: /<*> dest: /<*>
BLOCK* NameSystem.allocateBlock: <*> <*>
Verification succeeded for <*>
Deleting block <*> file <*>
<*> Served block <*> to /<*>
<*>:Got exception while serving <*> to /<*>:
BLOCK* NameSystem.delete: <*> is added to invalidSet of <*>
<*> Starting thread to transfer block <*> to <*>
BLOCK* ask <*> to delete <*>
Received block <*> src: /<*> dest: /<*> of size 67108864
BLOCK* ask <*> to delete <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <

In [62]:
for i in range(len(event2)):
  a = df_groundtruth['EventTemplate'][df_groundtruth['EventId']==event2[i]]
  for j in a:
    print(j)
    break

PacketResponder <*> for block blk_<*> terminating
BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*>
Received block blk_<*> of size <*> from /<*>
Receiving block blk_<*> src: /<*>:<*> dest: /<*>:<*>
BLOCK* NameSystem.allocateBlock: /<*>/part-<*>. blk_<*>
Verification succeeded for blk_<*>
Deleting block blk_<*> file /<*>/blk_<*>
<*>:<*> Served block blk_<*> to /<*>
<*>:<*>:Got exception while serving blk_<*> to /<*>:
BLOCK* NameSystem.delete: blk_<*> is added to invalidSet of <*>:<*>
<*>:<*> Starting thread to transfer block blk_<*> to <*>:<*>
BLOCK* ask <*>:<*> to delete  blk_<*>
Received block blk_<*> src: /<*>:<*> dest: /<*>:<*> of size <*>
BLOCK* ask <*>:<*> to replicate blk_<*> to datanode(s) <*>:<*>


In [64]:
df_parsedlog[df_parsedlog['EventId']==event[15]]

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
1900,1901,81111,91733,19,INFO,dfs.FSNamesystem,BLOCK* ask 10.251.126.5:50010 to delete blk_-...,415a1760,BLOCK* ask <*> to delete <*> <*> <*> <*> <*> <...,[]
